## Introduction

**Objective**

This script downloads news articles from the society section of the website of the Tunisian daily newspaper 'Assarih', 'https://www.assarih.com/r4-مجتمع'

**Pipeline**

1. Harvest list of article urls from web partal's main (front) web page
2. Download article web pages of harvested urls (html content)
3. Scrape textual content and metadata (date and title) of each downloaded article

In [8]:
import pandas as pd
import time

In [4]:
from scrapy import Selector

In [5]:
import requests

*****

## Harvest article urls from web portal's main page

In [6]:
## configuration parameters specific to the Assarih newspaper

#BASE_URL = 'https://www.assarih.com/r4-%D9%85%D8%AC%D8%AA%D9%85%D8%B9'   ##ENCODED FORM OF BASE URL
BASE_URL = 'https://www.assarih.com/r4-مجتمع'
xpath_title = "//div[@class='article']//div[@class='article-title']/a/text()"
xpath_link = "//div[@class='article']//div[@class='article-title']/a/@href"
xpath_date = "//div[@class='article']//div[@class='article-title']/p/text()"
xpath_content = "//div[@class='last-article']//text()"

In [9]:
# for each article extract its (hyperlink, title, date)

res = []
start_time = time.time()

for i in range(1,500):
    url = BASE_URL+'?page={}'.format(i)
    html = requests.get(url).content
    sel = Selector(text = html)
    u = sel.xpath(xpath_title).extract()
    v = sel.xpath(xpath_link).extract()
    w = sel.xpath(xpath_date).extract()
    z = [y for y in [x.strip() for x in w] if y!='']
    if len(u)!=len(v):
        print("warning: u and v are of different lengths.")
    if len(u)!=len(z):
        print("warning: u and z are of different lengths.")
    res = res + list(zip(u,v,z))
    if (i%50)==0:
        print(i)
    #print(i,len(u),len(v),len(z))

print(f"Harvested {len(res)} urls.")
print("--- %s minutes elapsed ---" % ((time.time() - start_time)/60))

50
100
150
200
250
300
350
400
450
Harvested 7485 urls.
--- 5.308672106266021 minutes elapsed ---


****

### Store harvested urls in database

In [10]:
data_df_new = pd.DataFrame(res, columns=['title','hyperlink','date'])
data_df_new.shape

(7485, 3)

In [11]:
data_df_new.head()

,title,hyperlink,date
0,جريمة بأطوار غريبة..استيقظ الزوج ليلا فوجد زو...,a161665--جريمة-بأطوار-غريبة..استيقظ-الزوج-ليلا...,26 فيفري 2020
1,يحدث في تونس : تلميذ يعتدي على زميلته بـ«لام» ...,a161660-يحدث-في-تونس---تلميذ-يعتدي-على-زميلته-...,26 فيفري 2020
2,حملة أمنية في بنزرت (صور),a161656-حملة-أمنية-في-بنزرت--صور-,26 فيفري 2020
3,"تزامنا مع التخوفات من ""كورونا"" : وفاة جديدة با...",a161648-تزامنا-مع-التخوفات-من--كورونا----وفاة-...,26 فيفري 2020
4,تورط في 10 قضايا تحويل وجهة : سفاح النساء بالا...,a161647-تورط-في-10-قضايا-تحويل-وجهة---سفاح-الن...,26 فيفري 2020


In [12]:
data_df_new.tail()

,title,hyperlink,date
7480,جزائريون بالطارف منعوا 20 سيارة تونسية من العب...,a129489-جزائريون-بالطارف-منعوا-20-سيارة-تونسية...,12 مارس 2019
7481,أميرها إمام جامع : محاكمة كتيبة إرهابية خططت ل...,a129488-أميرها-إمام-جامع---محاكمة-كتيبة-إرهابي...,12 مارس 2019
7482,خاص/ القصرين : معركة بين دوريات أمنية ومروجي م...,a129487-خاص--القصرين---معركة-بين-دوريات-أمنية-...,12 مارس 2019
7483,في البحيرة : أثيوبية تضع في فم مولودها كمية من...,a129486-في-البحيرة---أثيوبية-تضع-في-فم-مولودها...,12 مارس 2019
7484,في المنستير: ينكّل بوالدته ويضعها على السرير ب...,a129485-في-المنستير--ينكّل-بوالدته-ويضعها-على-...,12 مارس 2019


#### remove duplicate urls

In [51]:
data_df_new.hyperlink.nunique()<data_df_new.shape[0]

False

In [19]:
if data_df_new.hyperlink.nunique()<data_df_new.shape[0]:
    data_df_new.drop_duplicates(subset='hyperlink',inplace=True)
    data_df_new.reset_index(drop=True,inplace=True)
    data_df_new.shape

(7485, 3)

In [52]:
assert (data_df_new.index[-1]+1)==data_df_new.shape[0]

In [53]:
data_df_new.shape

(7485, 6)

#### cleanup hyperlink field
convert from relative to absolute url

In [21]:
BASE_URL0 = 'https://www.assarih.com/'

In [22]:
## convert to absolute url

data_df_new['hyperlink'] = data_df_new.hyperlink.map(lambda url: BASE_URL0+url)
data_df_new.head()

,title,hyperlink,date
0,جريمة بأطوار غريبة..استيقظ الزوج ليلا فوجد زو...,https://www.assarih.com/a161665--جريمة-بأطوار-...,26 فيفري 2020
1,يحدث في تونس : تلميذ يعتدي على زميلته بـ«لام» ...,https://www.assarih.com/a161660-يحدث-في-تونس--...,26 فيفري 2020
2,حملة أمنية في بنزرت (صور),https://www.assarih.com/a161656-حملة-أمنية-في-...,26 فيفري 2020
3,"تزامنا مع التخوفات من ""كورونا"" : وفاة جديدة با...",https://www.assarih.com/a161648-تزامنا-مع-التخ...,26 فيفري 2020
4,تورط في 10 قضايا تحويل وجهة : سفاح النساء بالا...,https://www.assarih.com/a161647-تورط-في-10-قضا...,26 فيفري 2020


In [23]:
data_df_new.tail()

,title,hyperlink,date
7480,جزائريون بالطارف منعوا 20 سيارة تونسية من العب...,https://www.assarih.com/a129489-جزائريون-بالطا...,12 مارس 2019
7481,أميرها إمام جامع : محاكمة كتيبة إرهابية خططت ل...,https://www.assarih.com/a129488-أميرها-إمام-جا...,12 مارس 2019
7482,خاص/ القصرين : معركة بين دوريات أمنية ومروجي م...,https://www.assarih.com/a129487-خاص--القصرين--...,12 مارس 2019
7483,في البحيرة : أثيوبية تضع في فم مولودها كمية من...,https://www.assarih.com/a129486-في-البحيرة---أ...,12 مارس 2019
7484,في المنستير: ينكّل بوالدته ويضعها على السرير ب...,https://www.assarih.com/a129485-في-المنستير--ي...,12 مارس 2019


#### parse date field

In [24]:
import dateparser

**Scratch Pad**

In [25]:
dateparser.parse('12/12/12')

datetime.datetime(2012, 12, 12, 0, 0)

In [26]:
dateparser.parse('2015, Ago 15, 1:08 pm', languages=['pt', 'es'])

datetime.datetime(2015, 8, 15, 13, 8)

In [27]:
x = data_df_new.date.iloc[-1]
print(x)
y = dateparser.parse(x)
type(y)

12 مارس 2019


datetime.datetime

In [28]:
y

datetime.datetime(2019, 3, 12, 0, 0)

In [29]:
y.month,y.day

(3, 12)

In [30]:
pd.to_datetime(y)

Timestamp('2019-03-12 00:00:00')

In [31]:
x = data_df_new.date.iloc[-1000]
print(x)
y = dateparser.parse(x,languages=['ar'])
y==None

02 ماي 2019


True

**End Scratch Pad**

Remark about method using the `dateparser` library :
- This method failed for many dates and produced None values; for example '29 أفريل 2019'. Some dates were parsed correctly such as '09 مارس 2019'.
- Apparently this is because some month names were not recognized by the dateparser library.
- In fact, month names used in Tunisia --and North Africa-- are Arabic transliterations of French month names, a legacy of French colonialism.
- Most likely the `dateparser` library uses month names used in Arab countries of the Levant and Gulf regions ...

In [32]:
## First method: using the dateparser library
## took a long while to run (several minutes)

# %time data_df_new['date_dt'] = data_df_new.date.map(dateparser.parse)

In [33]:
## therefore I decided to write my own date parsing method, combined with Pandas's to_datetime method.

tunisian_month_to_number_dict = {'جانفي':1,
 'ديسمبر':12,
 'أكتوبر':10,
 'نوفمبر':11,
 'أوت':8,
 'ماي':5,
 'جويلية':7,
 'سبتمبر':9,
 'فيفري':2,
 'جوان':6,
 'أفريل':4,
 'مارس':3}

df1 = data_df_new.date.str.findall('(\d{1,2})(.*)(20\d\d)').map(lambda x: x[0])
df2 = pd.DataFrame(df1.tolist(),columns=['day','month','year'])
df2['month_nb'] = df2.month.str.strip().map(tunisian_month_to_number_dict)

In [34]:
df2['date_dt'] = df2.apply(lambda row: pd.to_datetime(' '.join([str(row.month_nb),row.day,row.year])), axis=1)
df2.set_index(data_df_new.index,inplace=True)
df2.tail()

,day,month,year,month_nb,date_dt
7480,12,مارس,2019,3,2019-03-12
7481,12,مارس,2019,3,2019-03-12
7482,12,مارس,2019,3,2019-03-12
7483,12,مارس,2019,3,2019-03-12
7484,12,مارس,2019,3,2019-03-12


In [35]:
data_df_new['date_dt'] = df2['date_dt']
data_df_new.head()

,title,hyperlink,date,date_dt
0,جريمة بأطوار غريبة..استيقظ الزوج ليلا فوجد زو...,https://www.assarih.com/a161665--جريمة-بأطوار-...,26 فيفري 2020,2020-02-26
1,يحدث في تونس : تلميذ يعتدي على زميلته بـ«لام» ...,https://www.assarih.com/a161660-يحدث-في-تونس--...,26 فيفري 2020,2020-02-26
2,حملة أمنية في بنزرت (صور),https://www.assarih.com/a161656-حملة-أمنية-في-...,26 فيفري 2020,2020-02-26
3,"تزامنا مع التخوفات من ""كورونا"" : وفاة جديدة با...",https://www.assarih.com/a161648-تزامنا-مع-التخ...,26 فيفري 2020,2020-02-26
4,تورط في 10 قضايا تحويل وجهة : سفاح النساء بالا...,https://www.assarih.com/a161647-تورط-في-10-قضا...,26 فيفري 2020,2020-02-26


In [36]:
data_df_new.tail()

,title,hyperlink,date,date_dt
7480,جزائريون بالطارف منعوا 20 سيارة تونسية من العب...,https://www.assarih.com/a129489-جزائريون-بالطا...,12 مارس 2019,2019-03-12
7481,أميرها إمام جامع : محاكمة كتيبة إرهابية خططت ل...,https://www.assarih.com/a129488-أميرها-إمام-جا...,12 مارس 2019,2019-03-12
7482,خاص/ القصرين : معركة بين دوريات أمنية ومروجي م...,https://www.assarih.com/a129487-خاص--القصرين--...,12 مارس 2019,2019-03-12
7483,في البحيرة : أثيوبية تضع في فم مولودها كمية من...,https://www.assarih.com/a129486-في-البحيرة---أ...,12 مارس 2019,2019-03-12
7484,في المنستير: ينكّل بوالدته ويضعها على السرير ب...,https://www.assarih.com/a129485-في-المنستير--ي...,12 مارس 2019,2019-03-12


In [37]:
## double-check that the month and day were not swapped somehow

data_df_new.date_dt.tail().map(lambda x: x.month)

7480    3
7481    3
7482    3
7483    3
7484    3
Name: date_dt, dtype: int64

In [38]:
data_df_new.dtypes

title                object
hyperlink            object
date                 object
date_dt      datetime64[ns]
dtype: object

****

## Download text content from harvested articles' urls

In [39]:
import os.path
#from os import path

In [40]:
data_folder_path = '../data/'
outputfile_base_name = 'www.assarih.com'

In [41]:
import re

start_time = time.time()
filenames_list = []
content_list = []

for i,x in data_df_new.iterrows():
    title,url = x.title,x.hyperlink
    # check if record already exists
    title_ = re.sub('\W','_',title[0:70])
    filename = outputfile_base_name + "__" + title_ + '.txt'
    filenames_list.append(filename)
    filename = data_folder_path + filename
    if not(os.path.exists(filename)):
        # download raw html page then extract text content
        html = requests.get(url).content
        sel = Selector(text = html)
        u = sel.xpath(xpath_content).extract()
        v = ' '.join([x.strip() for x in u]).strip()
        if len(v)>0:
            # write content to a text file
            with open(filename, 'w', encoding='utf8') as f:
                f.write(url+'\n')
                f.write(v)
    else:
        with open(filename, 'r', encoding='utf8') as f:
            v = f.read()
    content_list.append(v)
    if (i%100)==0:
        print(i)

print("--- %s minutes elapsed ---" % ((time.time() - start_time)/60))

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
--- 1.5710351149241129 minutes elapsed ---


In [42]:
assert data_df_new.shape[0]==len(filenames_list)
assert data_df_new.shape[0]==len(content_list)

In [43]:
content_list[0:5]

['googletag.cmd.push(function() { googletag.display(\'div-gpt-ad-1546942376253-0\'); });    هي جريمة أخرى ولكن بأطوار وتطورات غريبة تلك التي شهدتها مدينة الجيزة، وذلك بعد أن أقدم زوج بالتعاون مع والده على قتل زوجته، وتم إيقافهما على الفور من قبل وحدات الأمن ليزعم الزوج أنه قتلها بعدما شاهدها تمارس الجنس مع والده! وأوضح الزوج المتهم في أقواله أمام النيابة العامة، إنه هناك خلافات شبه مستمرة بينه وبين زوجته؛ بسبب اكتشافه سوء سلوكها.  أضاف المتهم في ذات الأقوال أنه يقيم صحبة زوجته وأولادهما وأولاده من زوجة سابقة في شقة واحدة رفقة والده البالغ من العمر 70 عامًا؛ حيث يقيم هو وزوجته والأطفال في غرفة واحدة، والغرفة الأخرى يقيم بها والده؛ وبسبب هطول الأمطار المستمرة خلال الأيام الماضية على المدينة اضطر المتهم لترك غرفته، والنوم جميعًا برفقة والده على فراش واحد، مستخدمين غطاء واحد ليقيهم من البرد. واصل المتهم اعترافاته، مشيرا إلى أنه استيقظ ليلًا على أصوات وهمهمات ليتفاجأ بزوجته ووالده اللذان كانا ينامان بجانبه وهما بصدد ممارسة الجنس؛ وعندما شعرا باستيقاظه ادعيا النوم، فقام الزوج واصطحب الأطفال 

In [44]:
data_df_new['filename'] = pd.Series(filenames_list,index=data_df_new.index)
data_df_new['raw_text'] = pd.Series(content_list,index=data_df_new.index)

In [45]:
data_df_new.head()

,title,hyperlink,date,date_dt,filename,raw_text
0,جريمة بأطوار غريبة..استيقظ الزوج ليلا فوجد زو...,https://www.assarih.com/a161665--جريمة-بأطوار-...,26 فيفري 2020,2020-02-26,www.assarih.com___جريمة_بأطوار_غريبة__استيقظ_ا...,googletag.cmd.push(function() { googletag.disp...
1,يحدث في تونس : تلميذ يعتدي على زميلته بـ«لام» ...,https://www.assarih.com/a161660-يحدث-في-تونس--...,26 فيفري 2020,2020-02-26,www.assarih.com__يحدث_في_تونس___تلميذ_يعتدي_عل...,googletag.cmd.push(function() { googletag.disp...
2,حملة أمنية في بنزرت (صور),https://www.assarih.com/a161656-حملة-أمنية-في-...,26 فيفري 2020,2020-02-26,www.assarih.com__حملة_أمنية_في_بنزرت__صور_.txt,https://www.assarih.com/a158092-حملة-أمنية-في-...
3,"تزامنا مع التخوفات من ""كورونا"" : وفاة جديدة با...",https://www.assarih.com/a161648-تزامنا-مع-التخ...,26 فيفري 2020,2020-02-26,www.assarih.com__تزامنا_مع_التخوفات_من__كورونا...,googletag.cmd.push(function() { googletag.disp...
4,تورط في 10 قضايا تحويل وجهة : سفاح النساء بالا...,https://www.assarih.com/a161647-تورط-في-10-قضا...,26 فيفري 2020,2020-02-26,www.assarih.com__تورط_في_10_قضايا_تحويل_وجهة__...,googletag.cmd.push(function() { googletag.disp...


In [46]:
data_df_new.tail()

,title,hyperlink,date,date_dt,filename,raw_text
7480,جزائريون بالطارف منعوا 20 سيارة تونسية من العب...,https://www.assarih.com/a129489-جزائريون-بالطا...,12 مارس 2019,2019-03-12,www.assarih.com__جزائريون_بالطارف_منعوا_20_سيا...,https://www.assarih.com/a129489-جزائريون-بالطا...
7481,أميرها إمام جامع : محاكمة كتيبة إرهابية خططت ل...,https://www.assarih.com/a129488-أميرها-إمام-جا...,12 مارس 2019,2019-03-12,www.assarih.com__أميرها_إمام_جامع___محاكمة_كتي...,https://www.assarih.com/a129488-أميرها-إمام-جا...
7482,خاص/ القصرين : معركة بين دوريات أمنية ومروجي م...,https://www.assarih.com/a129487-خاص--القصرين--...,12 مارس 2019,2019-03-12,www.assarih.com__خاص__القصرين___معركة_بين_دوري...,https://www.assarih.com/a129487-خاص--القصرين--...
7483,في البحيرة : أثيوبية تضع في فم مولودها كمية من...,https://www.assarih.com/a129486-في-البحيرة---أ...,12 مارس 2019,2019-03-12,www.assarih.com__في_البحيرة___أثيوبية_تضع_في_ف...,https://www.assarih.com/a129486-في-البحيرة---أ...
7484,في المنستير: ينكّل بوالدته ويضعها على السرير ب...,https://www.assarih.com/a129485-في-المنستير--ي...,12 مارس 2019,2019-03-12,www.assarih.com__في_المنستير__ينك_ل_بوالدته_وي...,https://www.assarih.com/a129485-في-المنستير--ي...


****

## Store data frame in pickle file

In [47]:
pickle_filepath = '../pickle/articles_db.pickle'
csv_filepath = '../pickle/articles_db.csv'

In [48]:
import pickle

In [49]:
if os.path.exists(pickle_filepath):
    print(pickle_filepath,'already exists.')
    with open(pickle_filepath, 'rb') as f:
        data_df_0 = pickle.dump(f)
        print(data_df_0.shape,'existing records.')
        data_df = pd.concat([data_df_0,data_df_new],axis=0)
else:
    data_df = data_df_new

print(data_df.shape,'records in updated database.')

(7485, 6) records in updated database.


In [54]:
data_df_new.hyperlink.nunique()<data_df_new.shape[0]

False

In [ ]:
# remove duplicates

if data_df_new.hyperlink.nunique()<data_df_new.shape[0]:
    data_df_new.drop_duplicates(subset='hyperlink',inplace=True)
    data_df_new.shape

In [79]:
with open(pickle_filepath, 'wb') as f:
    pickle.dump(data_df, f)

In [80]:
# meta_data_df.to_csv(csv_filepath,index=True,encoding="utf8")